In [2]:
import os
os.sys.path.append("/Users/mocquin/MYLIB10/MODULES/physipy/tests")
os.sys.path.append("/Users/mocquin/MYLIB10/MODULES/physipy/")

This [Jupyter](http://jupyter.org/) notebook is an introduction to the quantity module of the [physipy](https://github.com/mocquin/physipy) package.  
This module is located in [https://github.com/mocquin/physipy/blob/master/physipy/quantity/dimension.py](https://github.com/mocquin/physipy/blob/master/physipy/quantity/quantity.py).

# Introduction 

The quantity module is the true module where most of the physipy package computation appends.

This module defines a Quantity class, which is basically the association of a mathematical value (scalar or array), with a a Dimension.

# Imports

In [9]:
import numpy as np

import physipy.quantity
from physipy.quantity import Quantity, Dimension

# The Quantity class

## Creation through init
A Quantity object is created with a mathematical value and a Dimension object. You can also add a favorit unit (which is only used for display), and a symbol (which is only used to make this quantity a favorite unit):
 - value : value must be a scalar or array
 - dimension : Dimension object
 - favunit : a favourit unit is a Quantity object whose symbol attribute is defined. Defaults to None
 - symbol : string, defaults to "\<UndefinedSymbol>"

Basic declaration only involve a value and a dimension : 

In [18]:
# remember the available dimensions are defined in physipy.quantity.dimension.SI_UNIT_SYMBOL
# print(physipy.quantity.dimension.SI_UNIT_SYMBOL)

# basic declaration
a = Quantity(12, Dimension("M"))
b = Quantity(0.2, Dimension("L"))
c = Quantity(np.array([1,2,3]), Dimension({"J":2,"theta":-2}))

for q in [a, b, c]:
    print("Quantity :", q)
    print("value :", q.value)
    print("dimension :", q.dimension)


Quantity : 12 kg
value : 12
dimension : M
Quantity : 0.2 m
value : 0.2
dimension : L
Quantity : [1 2 3] cd**2/K**2
value : [1 2 3]
dimension : J**2/theta**2


If you want to make this Quantity a favorite unit for another Quantity, you must define a symbol. Note that a symbol is only used for string representation. It doesn't change the data structure. Here we define custom Quantitys, with intelligent symbols.

In [25]:
# declaration with symbol
# the symbol attribute is only used when the quantity is the favunit of another quantity

# NOTE : this is a dummy example, the real gramm and kilometers are redefined 
# prorperly by the package elsewhere.
d = Quantity(0.001, Dimension("M"), symbol="g")
e = Quantity(1000, Dimension("L"), symbol="km")

We can then use these Quantity to define a favorite unit for another quantity.

In [34]:
# declaration with favunit : the favunit argument must be another quantity whose symbol attribute is defined

# here, we defined twice the same quantity (1 kg), but with 2 different favunits
# with no favunit : the default representation will use the SI unit (ie kg)
f = Quantity(1, Dimension("M"))
# with a favunit : instead of using the default SI unit, the Quantity will be displayed with the gramm
g = Quantity(1, Dimension("M"), favunit=d)

print("With no favunit (using default SI unit :")
print(f)
print("With a favunit that is worth 1g=0.001 kg : ")
print(g)

With no favunit (using default SI unit :
1 kg
With a favunit that is worth 1g=0.001 kg : 
1000.0 g


See also the following code : 

In [35]:
# The favunit we will use
print(d)
print(d.symbol)
# The quantity without favunit
print(f)
# Changing the quantity's favunit
f.favunit = d
# hence changing the representation
print(f)

0.001 kg
g
1 kg
1000.0 g


## Usefull methods

### dimensionless quantities
A dimensionless quantity is possible.

In [36]:
print(a.is_dimensionless())
print(Quantity(1,Dimension(None)).is_dimensionless())

False
True


### remove dimension if possible
If a quantity is dimensionless, it can be casted back to its value

In [47]:
q = Quantity(12.1234,Dimension(None))
print(type(q))
print(type(q.value))
print(q)
q_casted = q.remove_dimension_if_dimensionless()
print(type(q_casted))
print(q_casted)

<class 'physipy.quantity.quantity.Quantity'>
<class 'float'>
12.1234
<class 'float'>
12.1234


## Basic operations

### Addition, soustraction \__add__ __sub__
Pour additionner ou soustraire une quantité, l'opérande doit être une quantité. Ainsi, l'opérande est convertie en quantité : 
 - si c'est une quantité, on ne change rien
 - si c'est un scalaire ou un tableau, on le convertie en quantité sans dimension
 
Pour additionner ou sosutraire 2 quantités, leurs dimensions doivent être les mêmes. Si ce n'est pas le cas, on lève une exception.
Lorsqu'on soustrait ou additionne 2 quantité :
 - valeurs : leurs valeurs s'ajoute
 - dimension : la dimension résultante est la même que celle des opérandes
 - unité favorite : aucune, valeur par défaut
 - symbole : aucune, valeur par défaut

In [48]:
# a = 12kg, b = 12.334m
k = a + a
print("---- Inspection de k")
print(k.value)
print(k.dimension)
print(k.favunit)
print(k.symbol)

l = b - b
print("---- Inspection de l")
print(l.value)
print(l.dimension)
print(l.favunit)
print(l.symbol)

#m = a + b # --> Lève une erreur de dimension

---- Inspection de k
24
M
None
UndefinedSymbol
---- Inspection de l
0.0
L
None
UndefinedSymbol


## Multiplication , division \__mul\__ et \__div\__

Pour multiplier ou diviser, on converti l'opérande en dimension. Lorsqu'on multiplie ou divise 2 quantités:
 - valeur : les valeurs se multiplient/divisent
 - dimension : les dimensions se multiplient/divisent (cf méthodes de Dimension). Si la grandeur résultante est sans dimension, on lui retire via la méthode retire_unite_si_possible
 - unité favorite : valeur par défaut
 - symbole : valeur par défaut

In [49]:
# a = 12kg, b = 12.334m, c = [1,2,3]cd**2/K**2
n = a * b
print("---- Inspection de n")
print(n.value)
print(n.dimension)
print(n.favunit)
print(n.symbol)

o = b / c
print("---- Inspection de o")
print(o.value)
print(o.dimension)
print(o.favunit)
print(o.symbol)

print("---- Inspection de oo et ooo : dimension retirée")
oo = b / b
print(oo)
print(type(oo))
ooo = c / c 
print(ooo)
print(type(ooo))

---- Inspection de n
2.4000000000000004
L*M
None
UndefinedSymbol**2
---- Inspection de o
[0.2        0.1        0.06666667]
L*theta**2/J**2
None
1
---- Inspection de oo et ooo : dimension retirée
1.0
<class 'float'>
[1. 1. 1.]
<class 'numpy.ndarray'>


## Puissance \__pow\__
On peut élever une quantité en puissance par un réel :
 - valeur : élevé à la puissance
 - dimension : élevé à la puissance
 - unité favorite : valeur par défaut
 - symbole : valeur par défaut

In [50]:
p = c ** 2
print("---- Inspection de p")
print(p.value)
print(p.dimension)
print(p.favunit)
print(p.symbol)

q = c ** 2.5
print("---- Inspection de q")
print(q.value)
print(q.dimension)
print(q.favunit)
print(q.symbol)


---- Inspection de p
[1 4 9]
J**4/theta**4
None
UndefinedSymbol**2
---- Inspection de q
[ 1.          5.65685425 15.58845727]
J**5.0*theta**(-5.0)
None
UndefinedSymbol**2.5


## Comparaison eq, ne, lt, gt, le, ge

On peut comparer des quantités : elles doivent être de mêmes dimension. On compare ensuite leurs valeurs

In [56]:
cd = Quantity(1,Dimension("J"),symbol="cd")
K = Quantity(1,Dimension("theta"),symbol="K")

print(a < a)
print(a == a)
print(a == b)
print(a <= a)
print(c == b)
print(c == c)
print(c < c)
print(c < 2*cd**2/K**2)
print(a)
print(b)
print(c)

False
True
False
True
False
[ True  True  True]
[False False False]
[ True False False]
12 kg
0.2 m
[1 2 3] cd**2/K**2


## Conversions numériques abs, int, long, float, complex
Abs prend simplement la valeur absolue de la valeur.
Les autres fonctions évite de convertir une quantité dont la dimension n'est pas nulle.

In [57]:
r = abs(a)
#rr = int(a) --> lève une erreur : on ne peux pas convertir une quantité de dimension non nulle : ici kg
#rr = float(a)
#rr = long(a)
s = float(a/a) # résultat de la division sans dimension
t = float(Quantity(12,Dimension(None)))
print(s)
print(t)

1.0
12.0


## Conversions str et repr

In [58]:
a

<Quantity : 12 kg>

In [59]:
b

<Quantity : 0.2 m>

In [60]:
c

<Quantity : [1 2 3] cd**2/K**2>

In [61]:
d

<Quantity : 0.001 kg>

In [62]:
e

<Quantity : 1000 m>

In [63]:
print(f.value)
print(f.dimension)
print(f.favunit)
print(f.symbol)
f

1
M
0.001 kg
UndefinedSymbol


<Quantity : 1 kg>

In [64]:
m = Quantity(1,Dimension("L"),symbol="m")
mm = m/1000
mm.symbol = "mm"

aaa = 2*mm

aaa.favunit = mm
print(aaa)

2.0 mm


In [65]:
print(g.value)
print(g.dimension)
print(g.favunit.value)
print(g.favunit.dimension)
print(g.favunit.symbol)
print(g.favunit.value)
g.favunit

1
M
0.001
M
g
0.001


<Quantity : 0.001 kg>

In [66]:
kg = Quantity(1,Dimension("M"),symbol="kg")

g.favunit = kg/m
g

<Quantity : 1 kg>

In [67]:
#h # rsidu de l'import de dimension

In [68]:
print(i.value)
print(i.dimension)
print(i.favunit.value)
print(i.favunit.dimension)
print(i.favunit.symbol)
i.favunit

NameError: name 'i' is not defined

In [69]:
i

NameError: name 'i' is not defined

In [70]:
print(j.value)
print(j.dimension)
print(j.favunit.value)
print(j.favunit.dimension)
print(j.favunit.symbol)
j.favunit


NameError: name 'j' is not defined

In [71]:
j

NameError: name 'j' is not defined

In [72]:
print(k.value)
print(k.dimension)
#print(k.unite_favorite.valeur)
#print(k.unite_favorite.dimension)
#print(k.unite_favorite.symbole)
#k.unite_favorite
print(k.favunit)
print(k.symbol)
k

24
M
None
UndefinedSymbol


<Quantity : 24 kg>

In [73]:
print(l.value)
print(l.dimension)
print(l.favunit)
print(l.symbol)
l

0.0
L
None
UndefinedSymbol


<Quantity : 0.0 m>

In [74]:
print(n.value)
print(n.dimension)
print(n.favunit)
print(n.symbol)
n

2.4000000000000004
L*M
None
UndefinedSymbol**2


<Quantity : 2.4000000000000004 kg*m>

In [75]:
print(o.value)
print(o.dimension)
print(o.favunit)
print(o.symbol)
o

[0.2        0.1        0.06666667]
L*theta**2/J**2
None
1


<Quantity : [0.2        0.1        0.06666667] K**2*m/cd**2>

In [76]:
print(p.value)
print(p.dimension)
print(p.favunit)
print(p.symbol)
p

[1 4 9]
J**4/theta**4
None
UndefinedSymbol**2


<Quantity : [1 4 9] cd**4/K**4>

In [77]:
print(q.value)
print(q.dimension)
print(q.favunit)
print(q.symbol)
q

[ 1.          5.65685425 15.58845727]
J**5.0*theta**(-5.0)
None
UndefinedSymbol**2.5


<Quantity : [ 1.          5.65685425 15.58845727] K**(-5.0)*cd**5.0>

In [78]:
print(r.value)
print(r.dimension)
print(r.favunit)
print(r.symbol)
r

12
M
None
UndefinedSymbol


<Quantity : 12 kg>

In [79]:
#print(s.value)
#print(s.dimension)
#print(s.favunit)
#print(s.symbol)
s

1.0

In [80]:
#print(t.value)
#print(t.dimension)
#print(t.favunit)
#print(t.symbol)
#t

In [81]:
print(str(a))
print(str(b))
print(str(c))
print(str(d))
print(str(e))
print(str(f))
print(str(g))
#print(str(h))
print(str(i))
print(str(j))
print(str(k))
print(str(l))
#print(str(m)) pas défini
print(str(n))
print(str(o))
print(str(p))
print(str(q))
print(str(r))
print(str(s))
print(str(t))

12 kg
0.2 m
[1 2 3] cd**2/K**2
0.001 kg
1000 m
1000.0 g
1.0 kg/m*m


NameError: name 'i' is not defined

# Fonctions de quantité

## Initialisation des unités de bases

In [82]:
#initialise_unités_de_base() déjà fait plus haut

## Initialise multiples unitésde base

In [83]:
#initialise_sous_multiples_unités_de_base()

## Définition unité

In [84]:
#defineUnit("Hz", 1/s)

In [85]:
#Hz

# Fonctions utiles

## Interpolateur
L'interpolateur permet de calculer une valeur interpolée parmis un tableau de donnée, en conservant les unités. L'interpolateur fonctionne àla fois pour les entiers/float/tableau array et les quantités:

In [86]:
tab_x = [1, 2,3,4,5,6,7,8,9,10]
tab_y = [10,9,8,7,6,5,4,3,2,1]
variable = 4.5
Q_interp(variable, tab_x, tab_y)

NameError: name 'Q_interp' is not defined

In [42]:
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])
variable = 4.5
Q_interp(variable, tab_x, tab_y)

6.50 no-dimension

In [43]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])*m
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])
variable = 4.5*m
Q_interp(variable, tab_x, tab_y)

6.50 no-dimension

In [44]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])*m
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])*m**2
variable = 4.5*m
Q_interp(variable, tab_x, tab_y)

6.50 m**2

In [45]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])*m
variable = 4.5
Q_interp(variable, tab_x, tab_y)

6.50 m

Ne doivent pas fonctionner : 

In [46]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])*m
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])
variable = 4.5
#Q_interp(variable, tab_x, tab_y) --> Lève erreur

In [47]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])
variable = 4.5*m
#Q_interp(variable, tab_x, tab_y) --> lève erreur

In [48]:
m = Quantity(1,Dimension("L"),symbol="m")
tab_x = np.array([1, 2,3,4,5,6,7,8,9,10])*Quantity(1,Dimension("J"),symbol="cd")
tab_y = np.array([10,9,8,7,6,5,4,3,2,1])
variable = 4.5*m
#Q_interp(variable, tab_x, tab_y) #--> lève erreur

## Vectorizateur

In [49]:
mm = Quantity(0.001,Dimension("L"),symbol="mm")
f_cymm = 2/mm
f_cymm_array = np.array([1,2,3,4,5])/mm
def calcul_FTM_test_non_vectorize(fcymm):
    if fcymm > 1/Quantity(1,Dimension("L")):
        return 1-fcymm/(10*1/mm)
    else:
        return 0

In [50]:
calcul_FTM_test_non_vectorize(f_cymm)
#calcul_FTM_test_non_vectorize(f_cymm_array)# --> lève une erreur ; The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

0.8

In [51]:
mm = Quantity(0.001,Dimension("L"),symbol="mm")
f_cymm = 2/mm
f_cymm_array = np.array([1,2,3,4,5])/mm

@Q_vectorize
def calcul_FTM_test_oui_vectorize(fcymm):
    if fcymm > 1/Quantity(1,Dimension("L")):
        return 1-fcymm/(10*1/mm)
    else:
        return 0


In [52]:
calcul_FTM_test_oui_vectorize(f_cymm)
calcul_FTM_test_oui_vectorize(f_cymm_array)

[0.90 0.80 0.70 0.60 0.50] no-dimension

In [53]:
@Q_vectorize
def calcul_energie(lambda_):
    if lambda_ > 2*mm:    
        return 90*mm
    else:
        return 0.0*mm

In [54]:
calcul_energie(np.array([1,2,3,4,5])*mm)

[0.00 0.00 0.09 0.09 0.09] m

## Itérateur : getteur et setteur

In [55]:
u = np.array([1,2,3,4,5])*m
u.unite_favorite = Quantity(0.001,Dimension("L"),symbol="mm")

In [56]:
for val in u:
    print(val)

1.00 m
2.00 m
3.00 m
4.00 m
5.00 m


In [57]:
u[2] = 35*m
print(u)

[1.00 2.00 35.00 4.00 5.00] m


In [58]:
# --> lève une erreur, insérer une quantité doit être de même dimension
#u[4] = 35*Quantite(1,Dimension("kg"))   

## Linspace

In [59]:
m = Quantity(1,Dimension("L"),symbol="m")
print(m)
print(mm)
Q_linspace(2*mm,10*m)

1.00 m
0.00 m


[0.00 0.10 0.20 0.30 0.41 0.51 0.61 0.71 0.81 0.91 1.01 1.11 1.21 1.31 1.42 1.52 1.62 1.72 1.82 1.92 2.02 2.12 2.22 2.32 2.43 2.53 2.63 2.73 2.83 2.93 3.03 3.13 3.23 3.33 3.44 3.54 3.64 3.74 3.84 3.94 4.04 4.14 4.24 4.34 4.45 4.55 4.65 4.75 4.85 4.95 5.05 5.15 5.25 5.35 5.46 5.56 5.66 5.76 5.86 5.96 6.06 6.16 6.26 6.36 6.47 6.57 6.67 6.77 6.87 6.97 7.07 7.17 7.27 7.37 7.48 7.58 7.68 7.78 7.88 7.98 8.08 8.18 8.28 8.38 8.49 8.59 8.69 8.79 8.89 8.99 9.09 9.19 9.29 9.39 9.50 9.60 9.70 9.80 9.90 10.00] m

In [60]:
Q_linspace(2*mm,10*m,10)

[0.00 1.11 2.22 3.33 4.45 5.56 6.67 7.78 8.89 10.00] m

## Intégrateur

In [61]:
def test_func(x):
    return 2*x

Q_integrate(3*m,5*m,test_func)

16.00 m**2

In [62]:
import quantity
from quantity import Quantity, Dimension
import numpy as np

In [63]:
a = Quantity(np.array([1,2,3]), Dimension("L"))
b = a **1

In [64]:
print(a)
print(b)
print(type(a))
print(type(a.value))
print(a == b)

[1.00 2.00 3.00] m
[1.00 2.00 3.00] m
<class 'quantity.quantity.Quantity'>
<class 'numpy.ndarray'>
True


In [65]:
v1 = np.array([1,2,3])
v2 = np.array([4,5,6])
print(1**v1)
print(1**v2)
print(v1**v2)
print(v1**v2[:])

[1 1 1]
[1 1 1]
[  1  32 729]
[  1  32 729]


In [66]:
print(a)
print(a ** 1)
print(a ** 2)
#print(a ** np.array([1]))


[1.00 2.00 3.00] m
[1.00 2.00 3.00] m
[1.00 4.00 9.00] m**2


In [68]:
#np.cos(Quantity(1,Dimension(None)))

In [69]:
a = Quantity(1,Dimension("M"))

In [70]:
a.quantity_class_constant

1

In [71]:
a.quantity_class_constant = 2

In [72]:
a.quantity_class_constant

2

In [74]:
b = Quantity(2,Dimension("M"))
b.quantity_class_constant

1

In [2]:
from quantity import units

In [3]:
units

{'rad': 1.00 no-dimension,
 'sr': 1.00 no-dimension,
 'Hz': 1.00 1/s,
 'N': 1.00 kg*m/s**2,
 'Pa': 1.00 kg/(m*s**2),
 'J': 1.00 kg*m**2/s**2,
 'W': 1.00 kg*m**2/s**3,
 'C': 1.00 A*s,
 'V': 1.00 kg*m**2/(A*s**3),
 'F': 1.00 A**2*s**4/(kg*m**2),
 'Wb': 1.00 kg*m**2/(A*s**2),
 'T': 1.00 kg/(A*s**2),
 'H': 1.00 kg*m**2/(A**2*s**2),
 'lm': 1.00 cd,
 'lx': 1.00 cd/m**2,
 'Bq': 1.00 1/s}

In [4]:
T = units["T"]

In [5]:
T

1.00 kg/(A*s**2)

In [1]:
import quantity
from quantity import m, kg, s
from quantity import quantify, Quantity, Dimension

In [9]:
a = 2*m/s
km = 1000*m
km.symbol = 'km'
hr = 3600*s
hr.symbol = "hr"
km_hr = km/hr
print(a.to(km/hr))
print(a.to(km))
print(a.to(hr))


7.20 km/hr
0.00 km*1/s
5.56E-04 hr*m/s**2


In [13]:
a = 2*m/s
print(a)
a.favunit = km_hr
print(a)

2.00 m/s
7.20 km/hr


In [14]:
q = a/(km/hr)
q = quantify(q)
q.symbol = km.symbol / hr.symbol
print(str())

print(a/(km/hr))
print(a)


7.199999999999999
7.20 km/hr


In [3]:
import quantity
from quantity import Q_integrate, sqrt, cos, arctan
from quantity import m
from math import pi
import numpy as np


def angle_solide_disque_total(x,y,z,R_disque):
    def num(r_):
        return z*r_
    def denum(r_,theta2):
        return sqrt(r_**2 + x**2 + y**2 - 2 * r_ * sqrt(x**2 + y**2) * cos(2 * arctan( y / (x + sqrt(x**2 + y**2))) - theta2) + z**2)**(1/3)
    def fun(r_, theta2):
        return num(r_) / denum(r_, theta2)
    def int_1(r_):
        def func(theta_2):
            return fun(r_, theta_2)
        return Q_integrate(0, 2 * pi, func)
    # intégrer pour theta2 entre 0 et 2pi
    # Intégrer pour r_ entre 0 et R_disque
    return Q_integrate(0*m, R_disque, int_1)

x = y = z = 1 * m
R_disque = m

In [4]:
angle_solide_disque_total(x,y,z,R_disque)

2.85 m**2.66666666666667

In [6]:
import quantity
from quantity import dblquad
from quantity import m, kg

In [11]:
def func2D(x,y):
    return y*1*m * 1*m
        
dblquad(func2D, 0, 2, 0*kg, 2*kg)[0]
print(dblquad(func2D, 0, 2, 0*kg, 2*kg)[0].value)

3.9999999999999996


In [10]:
4*kg*m**2

4.00 kg*m**2